In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn
from datetime import datetime
from sklearn.model_selection import train_test_split as tt
from sklearn.metrics import accuracy_score as acc
%matplotlib inline

In [74]:
f = raw_input()
file = open(f, "r")
file1 = open("data_nn.csv", "w")
file1.write('"Time" "Ping Local" "Ping LocalApp-CorpApp" "Ping LocalApp-CorpDB" "Load Average LocalApp" "Load Average LocalDB" "Load Average CORPApp" "TNS ping LocalApp-LocalDB" "TNS ping LocalApp-CorpDB" "Class0" "Class1" "Class2" "Class3"\n')
hr = 12
m = 0
day = 0
for line in file:
    l = len(line)
    dat = list(line.split())
    if (dat[1] == 'x' or dat[5] == 'x'):
        line = "Day_"+str(day)+":"+str(hr)+":"+str(m)+" "
        for da in dat:
            if (da == 'x'):
                da = '9999999999'
            line+=da+" "
        line += "0 1 0 0\n"
    elif (dat[2] == 'x' or dat[7] == 'x'):
        line = "Day_"+str(day)+":"+str(hr)+":"+str(m)+" "
        for da in dat:
            if (da == 'x'):
                da = '9999999999'
            line+=da+" "
        line += "0 0 1 0\n"
    elif (dat[3] == 'x' or dat[4] == 'x' or dat[6] == 'x'):
        line = "Day_"+str(day)+":"+str(hr)+":"+str(m)+" "
        for da in dat:         
            if (da == 'x'):
                da = '9999999999'
            line+=da+" "
        line += "0 0 0 1\n"
    else:
        line = "Day_"+str(day)+":"+str(hr)+":"+str(m)+" "
        for da in dat:
            if (da == 'x'):
                da = '9999999999'
            line+=da+" "
        line += "1 0 0 0\n"
    file1.write(line)
    m=m+1
    if(m==60):
       m=0 
       hr=hr+1
       if (hr >= 24):
        hr = 0
        day+=1
file.close()
file1.close()

test3.csv


In [75]:
x = tf.placeholder(tf.float32,[None,8])
#weights
W = tf.Variable([[0.15463702, -1.21701765, 0.52780521, -1.4738549],
                [0.42833516, -0.04660674, 0.82068664, 0.09615696],
                [0.62039953, 0.93298858, -0.19275554, -1.06650257],
                [-0.05696351, -1.84619224, -0.93848866, -0.38569927],
                [-0.67261547, 0.19050605, -0.29760489, -0.62254256],
                [-1.1083765, -2.41506457, 0.25277814, -2.16972232],
                [0.35110912, 0.16467018, 2.88074422, -0.33394173],
                [1.98250139, -0.09985847, -0.04889834, -0.10028272]])
#W2 = tf.Variable(tf.random_normal([16,4]))
#biases
b = tf.Variable([0.13422392, -0.29879299, 0.13994822, 1.51326454])
#b2 = tf.Variable(tf.random_normal([4]))

#calculations
y_values = tf.add(tf.matmul(x,W),b)

y = tf.nn.softmax(y_values)

y_ = tf.placeholder(tf.float32,[None,4])

In [76]:
test = pd.read_csv('data_nn.csv',delimiter = ' ')
print (test)
time = test.ix[:,:1]
test = test.ix[:,1:9]

             Time  Ping Local  Ping LocalApp-CorpApp  Ping LocalApp-CorpDB  \
0      Day_0:12:0       0.276                   60.2                  57.1   
1      Day_0:12:1       0.324                   59.8                  56.7   
2      Day_0:12:2       0.441                   59.5                  56.5   
3      Day_0:12:3       0.433                   61.6                  56.9   
4      Day_0:12:4       0.347                   59.9                  56.5   
5      Day_0:12:5       0.366                   59.1                  56.7   
6      Day_0:12:6       0.378                   60.1                  56.8   
7      Day_0:12:7       0.411                   60.5                  56.5   
8      Day_0:12:8       0.357                   59.7                  56.7   
9      Day_0:12:9       0.257                   59.5                  56.2   
10    Day_0:12:10       0.317                   60.4                  56.3   
11    Day_0:12:11       0.399                   60.1            

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  This is separate from the ipykernel package so we can avoid doing imports until


In [77]:
inputX = test.as_matrix()
timeX = time.as_matrix()

In [78]:
with tf.Session() as sess:
    init = tf.initialize_all_variables()
    sess.run(init)
    result = sess.run(y,feed_dict={x:inputX})
rows = result.shape[0]

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [79]:
file = open('Time-Report.txt','w')
flag=0
c1 = 0
c2 = 0
c3 = 0
for i in range(rows):
    if result[i][1]==1:
        c1+=1
        if(c1==2):
            file.write("At "+str(timeX[i-1])+"Remote App is down\n")
        flag=1
    elif result[i][2]==1:
        c2+=1
        if(c2==2):
            file.write("At "+str(timeX[i-1])+"Remote DB is down\n")
        flag=2
    elif result[i][3]==1:
        c3+=1
        if(c3==2):
            file.write("At "+str(timeX[i-1])+"Local DB is down\n")
        flag=3
    else:
        if(c1>=2 and flag==1):
            file.write("At "+str(timeX[i])+"Remote App Back Up and Running\n\n")
        elif(c2>=2 and flag==2):
            file.write("At "+str(timeX[i])+"Remote DB Back Up and Running\n\n")
        elif(c3>=2 and flag==3):
            file.write("At "+str(timeX[i])+"Local DB Back Up and Running\n\n")
        flag=0
        c1=0
        c2=0
        c3=0
file.close()
        